In [ ]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
from changer_capacity.services.process import process
from changer_capacity.utils.get_capacity import get_min_capacity

In [5]:
base_capacity = get_min_capacity()
base_capacity

,name,capacity
0,school,250
1,kindergarten,80
2,hospital,110
3,polyclinic,50
4,pitch,10
...,...,...
76,golf_course,450
77,plant_gas_oil,264
78,railway_roundhouse,70
79,aeroway_terminal,320


In [ ]:
buildings_blocks = gpd.read_file('data/blocks.geojson')
acc_mx = pd.read_pickle('data/accessibility_matrix_drive.pickle')

In [ ]:
boundaries = ox.geocode_to_gdf('R337422', by_osmid=True)
local_crs = boundaries.estimate_utm_crs()

In [ ]:
service_type = 'school'
service = ox.features_from_polygon(boundaries.to_crs(4326).union_all(), tags={"amenity": service_type}).reset_index(drop=True).to_crs(local_crs)

In [ ]:
demand_per_1000 = 120
base_count = 250
m2_per_person = 14
k = 2

result, prov_df, prov_df_new = process(service_type, service, buildings_blocks, acc_mx, local_crs, demand_per_1000, base_count, m2_per_person, k)

In [ ]:
schools_new = result[['geometry','new_capacity']].rename({'new_capacity': 'capacity'}, axis=1)

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Filter the data to only include rows where capacity equals 250
filtered_data = result[result['capacity'] == 250.0]

plt.figure(figsize=(10, 6))
BINS = np.arange(200, 2001, 10)

# Plot histograms for both columns using the filtered data
plt.hist(result['capacity'], alpha=1, label='Capacity (250)', bins=BINS)
plt.hist(result['new_capacity'], alpha=0.5, label='New Capacity', bins=BINS)

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram: Capacity=250 vs New Capacity')
plt.legend(loc='upper right')

# Show the plot
plt.show()


In [ ]:
# %pip install folium mapclassify -qq

In [ ]:
from changer_capacity.visualisation.maps import make_blocks_capacity

m = make_blocks_capacity(
    buildings_blocks=buildings_blocks,
    prov_df_old=prov_df,
    prov_df_new=prov_df_new,
    id_col=None,            # или "block_id"
    prov_id_col_old=None,   # если id в prov_df_old — индекс
    prov_id_col_new=None,   # если id в prov_df_new — индекс
    capacity_col="capacity"
)
m

In [ ]:
# m.save("capacity_change_map.html")